In [1]:
import numpy as np
import pandas as pd
from scipy.special import gamma
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'


In [2]:
def n_ball_volume(n, radius):
    num = np.pi**(n / 2) * radius
    den = gamma((n / 2) + 1)
    return num / den


def origin_to_box_corner_dist(n, edge_len):
    edges = np.array([edge_len for x in range(1, n+1, 1)])
    return np.sqrt((edges**2).sum())
    

In [3]:
def run_experiment():
    
    dimensions = np.arange(1, 11, 1)
    
    corner_sphere_radius_list = []
    corner_sphere_volume_list = []
    
    hyper_box_edge_length_list = []
    hyper_box_max_length_list = []
    hyper_box_volume_list = []
    
    bounding_hyper_box_edge_length_list = []
    bounding_hyper_box_max_length_list = []    
    bounding_hyper_box_volume_list = []
    
    inner_sphere_radius_list = []
    inner_sphere_volume_list = []
    
    for dim in dimensions:
        
        corner_sphere_radius = 1
        
        hyper_box_edge_length = 1
        hyper_box_volume = hyper_box_edge_length ** dim
        hyper_box_max_length = origin_to_box_corner_dist(dim, hyper_box_edge_length)
        
        bounding_hyper_box_edge_length = 2 * hyper_box_edge_length
        bounding_hyper_box_volume = bounding_hyper_box_edge_length ** dim
        bounding_hyper_box_max_length = origin_to_box_corner_dist(dim, bounding_hyper_box_edge_length)

        inner_sphere_radius = hyper_box_max_length - corner_sphere_radius
        corner_sphere_volume = n_ball_volume(dim, corner_sphere_radius)
        inner_sphere_volume = n_ball_volume(dim, inner_sphere_radius)
        
        
        corner_sphere_radius_list.append(corner_sphere_radius)
        corner_sphere_volume_list.append(corner_sphere_volume)

        hyper_box_edge_length_list.append(hyper_box_edge_length)
        hyper_box_max_length_list.append(hyper_box_max_length)
        hyper_box_volume_list.append(hyper_box_volume)

        bounding_hyper_box_edge_length_list.append(bounding_hyper_box_edge_length)
        bounding_hyper_box_max_length_list.append(bounding_hyper_box_max_length)
        bounding_hyper_box_volume_list.append(bounding_hyper_box_volume)

        inner_sphere_radius_list.append(inner_sphere_radius)
        inner_sphere_volume_list.append(inner_sphere_volume)
        
    df = pd.DataFrame({
        'dimension': dimensions,
        'corner_sphere_radius': corner_sphere_radius_list,
        'corner_sphere_volume': corner_sphere_volume_list,
        'hyper_box_edge_length': hyper_box_edge_length_list,
        'hyper_box_max_length': hyper_box_max_length_list,
        'hyper_box_volume': hyper_box_volume_list,
        'bounding_hyper_box_edge_length': bounding_hyper_box_edge_length_list,
        'bounding_hyper_box_max_length': bounding_hyper_box_max_length_list,    
        'bounding_hyper_box_volume': bounding_hyper_box_volume_list,
        'inner_sphere_radius': inner_sphere_radius_list,
        'inner_sphere_volume': inner_sphere_volume_list,
    })
        
    return df





In [4]:
df = run_experiment()

In [5]:
df

,dimension,corner_sphere_radius,corner_sphere_volume,hyper_box_edge_length,hyper_box_max_length,hyper_box_volume,bounding_hyper_box_edge_length,bounding_hyper_box_max_length,bounding_hyper_box_volume,inner_sphere_radius,inner_sphere_volume
0,1,1,2.000000,1,1.000000,1,2,2.000000,2,0.000000,0.000000
1,2,1,3.141593,1,1.414214,1,2,2.828427,4,0.414214,1.301290
2,3,1,4.188790,1,1.732051,1,2,3.464102,8,0.732051,3.066407
3,4,1,4.934802,1,2.000000,1,2,4.000000,16,1.000000,4.934802
4,5,1,5.263789,1,2.236068,1,2,4.472136,32,1.236068,6.506401
5,6,1,5.167713,1,2.449490,1,2,4.898979,64,1.449490,7.490547
6,7,1,4.724766,1,2.645751,1,2,5.291503,128,1.645751,7.775790
7,8,1,4.058712,1,2.828427,1,2,5.656854,256,1.828427,7.421059
8,9,1,3.298509,1,3.000000,1,2,6.000000,512,2.000000,6.597018
9,10,1,2.550164,1,3.162278,1,2,6.324555,1024,2.162278,5.514163


### Key ideas at this point
#### Sphere packing
* The issue with high dimensions is that the max distance (from corner to corner of the hypercube) grows with each new dimension
* In terms of the sphere packing paradox, this means the following. As the max distance inside the hypercube grows, it means the corners are getting further away from eachother. This means that the unit spheres on the corners of the hypercube are getting further away from each other. This is creating more space for the inner sphere to grow, hence why it grows so quickly.

#### Volume
* In terms of volume, we have the best explanations [here](https://math.stackexchange.com/questions/15656/volumes-of-n-balls-what-is-so-special-about-n-5/15798#15798) and [here](https://math.stackexchange.com/questions/67039/why-does-the-volume-of-the-unit-sphere-go-to-zero)
* Essentially, when we go from n sphere to n + 1 sphere, we lose volume relative to a unit volume. A better way of putting that is that as we increase dimensions, a hypersphere is a worse approximation to the corresponding enclosing hypercube
* Going from a 1-d sphere to a 2-d sphere, we can think of sweeping the volume and scaling it by a factor of $\sqrt{1 - y^2}$
* Going from a 2-d sphere to a 3-d sphere, if we swept with a scale factor of 1 we would get a cylinder. But, based on the definition of a sphere (put mathematical def in here), we must again apply a scale factor of $\sqrt{1 - z^2}$. We can invision this as stacking pancakes of ever so slightly decreasing radius on top of eachother. See [here](https://photos.google.com/photo/AF1QipNXXzSUktvgV08GbLEf9cUevFNnkZ-WIdRc7k_4)
* Worth noting is that there are two effects at play here. At each dimension we reduce the amount of volume inside the hypersphere relative to the hypercube. Then, at the next increased dimension out hypersphere is already starting with less volume, and then we sweep it using a scaling term, reducing it even further.
* Remember that volume is usually described in terms of units$^d$ (for example, 3-d volume is described in units$^3$). This is all to say that when we talk about a single unit of volume in 3-d space, we are really referring to the space occupying a 1x1x1 cube. So, when we say that a unit sphere in 3-d has 4.188 units of volume, that is just to say that it contains the same amount of space as 4.188 1x1x1 cubes. The reason this is so important to keep in mind is that when talking about volume we are always implicitly making reference to a $\overbrace{1 \times 1 \times \dots \times 1}^\text{d}$ hypercube. So, if we say that the volume of a unit hypershere goes to 0 as $d \rightarrow \infty$, we are really saying that the _ratio_ of the volume of a $d$ dimensional hyper sphere to that of a $d$ dimensional unit hypercube goes to 0.

### Post structure:
* Section 1: Motivating Questions
    * Curse of dimensionality
    * Sphere packing
    * Radius of sphere inscribed in [-1, 1] hypercube stays at 1 as dimensions grow. Ideas to use [here](https://photos.google.com/photo/AF1QipP-5Wb2azhZUS_SQC0nxaGYWW2NRpkJFaWDg7XD), and pg 2 elementary introduction to convex geometry.
    * Euclidean distance in higher dimensions
        * [whiteboard](https://photos.google.com/search/_eAF1QipOQyc3orxK6jfimrO9nd~uN7nyUdy~uuIN8A_whiteboards/photo/AF1QipPxTzhpcYudipofWj5CYvbzuh3Bv0qoSurUGHS5)
    
    * n-ball volume problem
        * [scientific american article](https://www.americanscientist.org/sites/americanscientist.org/files/201110101628308738-2011-11CompSciHayes.pdf) 
* Section 2: Necessary background
    * Euclidean distance in 1, 2, 3, 4 + dimensions
    * Measure
    * Manifolds
    
* Section 3: Sphere Packing

* Section 4: Nball

* Section 5: Distance in high dimensions
    * All of mass is near shell
    * Why are things so far apart in eucldiean space in higher dimensions? The big thing to answer here is how does this relate to sphere packing, nball, Combinatorial approach? The reason things must become further apart is because volume grows? meaning things can be further apart? If you randomly sample a point it is likely to be in one of the corners (since volume of corners of hypercube grow, and these are far apart. Most of volume is in corners, hence most points are in corners, and corners are far apart.

    
* Appendix
    * Volume of hypersphere equation [video](https://www.youtube.com/watch?v=eEJtMFdkzzU)
        * KEY POINT: Make sure it is very clear how this equation returns a volume that is with respect to a unit of hyper volume. For example, in 2d the area of a circle is $\pi r^2 = 3.14...$ , when $r=1$. Now, implicit in that result is that the area has a _unit_ of $units^2$. Hence, $\pi r^2 = 3.14 \text{units}^2$. A single unit is defined to be a length of 1, hence $units^2$ is defined to be the area contained in a 1x1 box. So, saying that we have $\pi$ units$^2$ simply means that the amount of area contained in our unit circle is equivalent to that contained in a $1x1$ box. This ties in heavily with the ideas in [this comment](https://math.stackexchange.com/questions/67039/why-does-the-volume-of-the-unit-sphere-go-to-zero), namely that the unit that you measure with goes to $0$. 
        * So, the question could be asked: Is the denominator in the hypersphere equation related to the unit hypercube volume? How is that factored in? Isn't that just a value of 1?
        * [volume element spherical coordinates](https://www.youtube.com/watch?v=Bzlj2GpDsZw)


* Ideas to fit in somewhere:
    * A way to determine how "similar" two objects/sets are, we can do the following: Say we are trying to see how similar a hypercube is to a hypersphere. We can circumscribe the hypercube with the smallest hypersphere that will contain it. We can then inscribe the largest hypersphere that fits in the hypercube. We can then compare our two hyperspheres. If the geometry of the hypercube and hypersphere (abstractly) is similar, then our two hyperspheres should be similar (contain nearly the same volume/have the same geometrical properties). In the case that the geometry of the hypercube and hypersphere is identical, our hyperspheres should converge and be identical. On the other hand, if they have fundamentally different geometries, then our hyperspheres will not even remotely resemble each other. 

* Notes
    * Consider a combinatorial approach to explain why things blow up. Consider why euclidean distance may not be the be option anymore...
* Key takeaways
